In [4]:
import pandas as pd 
import numpy as np
import re


In [5]:
data = pd.read_csv('spam_ham_dataset.csv')


In [6]:
def split_subject_body(text):
    # Splitting by the first occurrence of "\r\n" which seems to separate the subject from the body
    parts = text.split("\r\n", 1)
    subject = parts[0].replace("Subject: ", "").strip() if len(parts) > 0 else ""
    body = parts[1].strip() if len(parts) > 1 else ""
    return subject, body

# Apply the function to the dataframe
data[['subject', 'body']] = data.apply(lambda row: pd.Series(split_subject_body(row['text'])), axis=1)

In [7]:
def count_words_clean_subject(subject):
    # Remove punctuation
    clean_subject = re.sub(r'[^\w\s]', '', subject)
    # Count the words
    word_count = len(clean_subject.split())
    return word_count

# Apply the function to the subject column to update the word count
data['subject_length'] = data['subject'].apply(count_words_clean_subject)
data.head()

,Unnamed: 0,label,text,label_num,subject,body,subject_length
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0,enron methanol ; meter # : 988291,this is a follow up to the note i gave you on ...,4
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0,"hpl nom for january 9 , 2001",( see attached file : hplnol 09 . xls )\r\n- h...,6
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0,neon retreat,"ho ho ho , we ' re around to that most wonderf...",2
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1,"photoshop , windows , office . cheap . main tr...",abasements darer prudently fortuitous undergon...,6
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0,re : indian springs,this deal is to book the teco pvr revenue . it...,3


In [8]:
def count_words_excluding_subject(text):
    subject_end_index = text.find("\n\n")
    if subject_end_index != -1:
        content = text[subject_end_index:].strip()
    else:
        content = text
        
    word_count = len(content.split())
    return word_count

In [9]:
data['body_length'] = data['text'].apply(count_words_excluding_subject)

In [10]:
data.head()

,Unnamed: 0,label,text,label_num,subject,body,subject_length,body_length
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0,enron methanol ; meter # : 988291,this is a follow up to the note i gave you on ...,4,67
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0,"hpl nom for january 9 , 2001",( see attached file : hplnol 09 . xls )\r\n- h...,6,23
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0,neon retreat,"ho ho ho , we ' re around to that most wonderf...",2,550
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1,"photoshop , windows , office . cheap . main tr...",abasements darer prudently fortuitous undergon...,6,48
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0,re : indian springs,this deal is to book the teco pvr revenue . it...,3,70
